In [21]:
import pandas as pd
import numpy as np

In [22]:
# csv paths
pathone = '../Project-1_Resources/aggr/VAERSDATA.csv'
paththree = '../Project-1_Resources/aggr/VAERSVAX.csv'

In [23]:
# Read in the VAERSDATA csv
df1 = pd.read_csv(pathone, low_memory=False)

# Drops rows with duplicate VAERS IDs, keeping the first, and ignoring the current the index 
data_df = df1.drop_duplicates(subset=['VAERS_ID'], keep='first', ignore_index=True)

# Filter data so only looking at reports from people 12 years of age or older
data_df = data_df.loc[data_df['AGE_YRS']>=12.00]

# Drop unneeded columns
data_df = data_df.drop(['CAGE_MO', 'CAGE_YR', 'SYMPTOM_TEXT', 'BIRTH_DEFECT', 'HISTORY', 
                        'SPLTTYPE','V_ADMINBY', 'V_FUNDBY', 'CUR_ILL', 'PRIOR_VAX',
                        'VAX_DATE', 'ONSET_DATE', 'LAB_DATA', 'OTHER_MEDS'],axis=1)

# Replace NaN with an empty cell
data_df = data_df.fillna('')

# Combine the two columns that list the date the form was completed (different names exist solely because 
# of the different VAERS forms used)
data_df['REPORT_DATE'] = data_df['RPT_DATE'] + data_df['TODAYS_DATE']

# Change Age in years to an integer
data_df["AGE_YRS"] = data_df["AGE_YRS"].astype("int64")

# Drop the two columns that we already merged into a new, single column
data_df = data_df.drop(['RPT_DATE','TODAYS_DATE'],axis=1)

data_df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,NUMDAYS,FORM_VERS,OFC_VISIT,ER_ED_VISIT,ALLERGIES,REPORT_DATE
1,413802,01/01/2011,CO,13,F,,,,Y,,,,,U,44,1,,,,01/01/2011
2,413803,01/01/2011,TN,67,F,,,,,,,,,U,0,1,,,,01/01/2011
3,413804,01/01/2011,MN,62,F,,,,,,,,,Y,0,1,,,,01/01/2011
6,413808,01/01/2011,WA,65,F,,,,,,,,,Y,1801,1,,,,01/01/2011
7,413830,01/01/2011,TN,76,M,,,,,Y,,Y,,N,9,1,,,,12/28/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798180,1442854,07/02/2021,FL,17,F,,,,,,,,,Y,0,2,,,,07/02/2021
798181,1442855,07/02/2021,RI,52,F,,,,,,,,,N,88,2,Y,,chlorhexidine scrub (rash),07/02/2021
798182,1442856,07/02/2021,CA,37,F,,,,,,,,,Y,0,2,,,,07/02/2021
798183,1442857,07/02/2021,DE,69,F,,,,,Y,,,,N,4,2,Y,,None known,07/02/2021


In [24]:
 # Read in the VAERSVAX csv
df3 = pd.read_csv(paththree, low_memory=False)

# Drops rows with duplicate VAERS IDs, keeping the first, and ignoring the current index
vax_df = df3.drop_duplicates(subset=['VAERS_ID'], keep='first', ignore_index=True)

# Drop unneeded columns
vax_df = vax_df.drop(['VAX_LOT', 'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME'], axis=1)

vax_df

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES
0,413801,PPV,MERCK & CO. INC.,1
1,413802,HPV4,MERCK & CO. INC.,UNK
2,413803,FLU3,CSL LIMITED,UNK
3,413804,PPV,UNKNOWN MANUFACTURER,1
4,413805,DTAPIPVHIB,SANOFI PASTEUR,1
...,...,...,...,...
798180,1442854,COVID19,PFIZER\BIONTECH,2
798181,1442855,COVID19,PFIZER\BIONTECH,1
798182,1442856,COVID19,PFIZER\BIONTECH,1
798183,1442857,COVID19,PFIZER\BIONTECH,2


In [25]:
# Drop unneeded columns
clean_data = pd.merge(data_df, vax_df, on="VAERS_ID", how="left")

clean_data

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,...,RECOVD,NUMDAYS,FORM_VERS,OFC_VISIT,ER_ED_VISIT,ALLERGIES,REPORT_DATE,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES
0,413802,01/01/2011,CO,13,F,,,,Y,,...,U,44,1,,,,01/01/2011,HPV4,MERCK & CO. INC.,UNK
1,413803,01/01/2011,TN,67,F,,,,,,...,U,0,1,,,,01/01/2011,FLU3,CSL LIMITED,UNK
2,413804,01/01/2011,MN,62,F,,,,,,...,Y,0,1,,,,01/01/2011,PPV,UNKNOWN MANUFACTURER,1
3,413808,01/01/2011,WA,65,F,,,,,,...,Y,1801,1,,,,01/01/2011,VARZOS,MERCK & CO. INC.,UNK
4,413830,01/01/2011,TN,76,M,,,,,Y,...,N,9,1,,,,12/28/2010,FLU3,NOVARTIS VACCINES AND DIAGNOSTICS,UNK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603024,1442854,07/02/2021,FL,17,F,,,,,,...,Y,0,2,,,,07/02/2021,COVID19,PFIZER\BIONTECH,2
603025,1442855,07/02/2021,RI,52,F,,,,,,...,N,88,2,Y,,chlorhexidine scrub (rash),07/02/2021,COVID19,PFIZER\BIONTECH,1
603026,1442856,07/02/2021,CA,37,F,,,,,,...,Y,0,2,,,,07/02/2021,COVID19,PFIZER\BIONTECH,1
603027,1442857,07/02/2021,DE,69,F,,,,,Y,...,N,4,2,Y,,None known,07/02/2021,COVID19,PFIZER\BIONTECH,2


In [26]:
# Export cleaned data to a csv file
clean_data.to_csv('Output/clean_data.csv')